In [ ]:
from main import *

start_time = time.time();
args = parser.parse_args("--num-processes 4 --epochs 4 --log-interval 100".split())

use_cuda = args.cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
dataloader_kwargs = {'pin_memory': True} if use_cuda else {}

torch.manual_seed(args.seed)
mp.set_start_method('spawn')

model = Net().to(device)
model.share_memory() # gradients are allocated lazily, so they are not shared here

processes = []
for rank in range(args.num_processes):
    p = mp.Process(target=train, args=(rank, args, model, device, dataloader_kwargs))
    # We first train the model across `num_processes` processes
    p.start()
    processes.append(p)
for p in processes:
    p.join()

elapsed_time = (time.time() - start_time) * 1000;

# Once training is complete, we can test the model
test(args, model, device, dataloader_kwargs)

print("Total training completion time is {} ms".format(int(round(elapsed_time))))

